In [130]:
import numpy as np
import pandas as pd
import igraph as ig

In [131]:
df = pd.read_csv("data/test.csv")

# convert to strings
df = df.astype(str)
# replace nans with blank string
df = df.applymap(lambda s: "" if s == "nan" else s)
# remove newlines
df = df.applymap(lambda s: s.rstrip('\n'))

# display data
df

,Full Name,Nickname,Father,Mother,Spouse,Children
0,Mother,Mom,Maternal Grandfather,Maternal Grandmother,Father,"Me, Brother, Sister"
1,Father,Dad,Paternal Grandfather,Paternal Grandmother,Mother,"Me, Brother, Sister"
2,Me,,Father,Mother,,
3,Sister,Sis,Father,Mother,,
4,Brother,Bro,Father,Mother,,
5,Paternal Grandfather,PGP,,,Paternal Grandmother,"Father, Ben"
6,Paternal Grandmother,PGM,,,Paternal Grandfather,"Father, Ben"
7,Maternal Grandfather,MGP,,,Maternal Grandmother,"Mother, Sue"
8,Maternal Grandmother,MGM,,,Maternal Grandfather,"Mother, Sue"
9,Ben,,Paternal Grandfather,Paternal Grandmother,,


In [168]:
# number of vertices
V = len(df)
# create index dictionary
di = dict()
for i in range(V):
    di[df["Full Name"][i]] = i
di[""] = -1

# create edges
E = [(source, di[dest]) for source in range(V) \
        for dest in list(map(lambda s: s.strip(), df.loc[source, "Children"].split(",")))]

# create graph
G = ig.Graph(directed = True)
G.add_vertices(V)
G.add_edges([e for e in E if e[1] != -1])

# visual styles
vs = {}
vs["vertex_label"] = df["Full Name"]
vs["layout"] = G.layout("rt")

In [190]:
children = G.degree(mode="out")

def height(node):
    if children[node] == 0:
        return -1
    else:
        return max(height(e[1]) for e in E if e[0] == node) + 1
    
def levelHelper(curr, node, level):
    if curr == -1:
        return 0
    
    if curr == node:
        return level
    
    downlevel = 0
    for e in E:
        if e[0] == curr:
            child = e[1]
            downlevel = levelHelper(child, node, level+1)
            if downlevel != 0:
                return downlevel
    return downlevel
    
def level(node):
    return levelHelper(di["Paternal Grandfather"], node, 1)

In [196]:
level(7)

0